In [ ]:
from mainv3 import SystemDesign
import matplotlib.pyplot as plt
from equations import JouybanAcreeModel
import numpy as np
from groups import ja_groups
import glob
from data_module import DataProcessor
import pandas as pd
import scipy.stats as stats
from equations import JouybanAcreeModel
from groups import ja_groups
from sklearn.metrics import mean_squared_error, r2_score


# Set up initial configurations for plots
plt.rcParams.update({
    'font.size': 12,          # Default font size
    'axes.labelsize': 14,     # Axis labels
    'axes.titlesize': 16,     # Subplot titles
    'xtick.labelsize': 12,    # X-axis tick labels
    'ytick.labelsize': 12,    # Y-axis tick labels
    'legend.fontsize': 12,    # Legend text
    'figure.titlesize': 18    # Figure title
})

In [ ]:
ja_model = JouybanAcreeModel()

In [ ]:
def calculate_metrics(weight_fractions: pd.Series, solubility_g_g: pd.Series, solvent_1_pure, solvent_2_pure, temperature, J0, J1, J2):
    predicted_solubility = ja_model.predict(
        weight_fractions,
        solvent_1_pure,
        solvent_2_pure,
        temperature,
        J0, J1, J2
    )

    # Calculate error metrics
    rmse = np.sqrt(mean_squared_error(weight_fractions, predicted_solubility))
    r2 = r2_score(solubility_g_g, predicted_solubility)
    mape = np.mean(np.abs((solubility_g_g - predicted_solubility) / solubility_g_g)) * 100
    
    return rmse, r2, mape, np.log(mape) 

In [ ]:
def AppendResults(system_load: SystemDesign):
    x,y = system_load.get_data_split_df()
    y_pred = system_load.predict_model(x)

    df = system_load.dataprocess.raw_data.merge(
        y_pred,
        left_index=True,
        right_index=True,
        suffixes=('','_pred')
    )
    
    results = []
    for i,row in df.iterrows():
        gn = int(row['group_index'])
        
        group = ja_groups[gn]
        CalculateMetrics = calculate_metrics(
            group['solvent_1_weight_fraction'],
            group['solubility_g_g'],
            row['solvent_1_pure'],
            row['solvent_2_pure'],
            row['temperature'],
            row['J0_pred'],
            row['J1_pred'],
            row['J2_pred']
        )
        
        results.append({
            "group_index": gn,
            "rmse": CalculateMetrics[0],
            "r2": CalculateMetrics[1],
            "mape": CalculateMetrics[2],
            "logmape": CalculateMetrics[3],
        })
    
    results = pd.DataFrame(results)
    
    df = df.merge(
        results,
        left_on='group_index',
        right_on='group_index',
        suffixes=('','_pred')
    )
    
    return df

In [ ]:
models_dir = '../../output/models'
model_files = glob.glob(f'{models_dir}/*.pkl')

models = {}
for model_file in model_files:
    system: SystemDesign = SystemDesign.load(model_file)
    # Extract model name from file path
    model_name = model_file.split('\\')[-1].replace('.pkl', '')
    
    models[model_name] = {
        'system': system,
        'results': AppendResults(system)
    }

In [ ]:
models_df = []
for key, values in models.items():
    model = values['results']
    
    log_diff_mean = abs(model['logmape'] - model['logmape_pred']).sum() / len(model['logmape'])
    log_diff_std = np.std(model['logmape'] - model['logmape_pred'])
    
    diff_mean = abs(model['mape'] - model['mape_pred']).sum() / len(model['mape'])
    
    models_df.append({
        'model': key,
        'log_diff_mean': log_diff_mean,
        'log_diff_std': log_diff_std,
        'diff_mean': diff_mean,
        'n': len(model['mape']),
    })

models_df = pd.DataFrame(models_df)

# Split the model column into its components
models_df[['model_type', 'combination', 'extra', 'features']] = models_df['model'].str.extract(r'(\w+)_system(?:_([a-z]+))?_(\d+)_(\d+)_features')

# Fill NaN values in combination column
models_df['combination'] = models_df['combination'].fillna('')

# Convert numeric columns to appropriate types
models_df['extra'] = models_df['extra'].astype(int)
models_df['features'] = models_df['features'].astype(int)

# Filter out rows where features value is 500
# models_df = models_df[models_df['features'] != 500]
# models_df = models_df[models_df['extra'] != 3]

models_df = models_df.sort_values(by='log_diff_mean', ascending=True)

# Display the dataframe with the new columns
models_df

In [ ]:
import numpy as np

import scipy.stats as stats

# Get the target model
target_model = models_df[models_df['model'] == 'xgb_system_ss_0_50_features']
diffs = target_model['log_diff_mean'].values[0]
std = target_model['log_diff_std'].values[0]
n = 10

# Calculate t-statistic for paired test
# Standard error of the mean (SEM)
sem = std / np.sqrt(n)

# t-statistic
t_stat = diffs / sem

# Degrees of freedom
df = n - 1

# Calculate the p-value using the t-distribution
# For two-tailed test (using numpy's implementation)
p_value = 2 * (1 - stats.t.cdf(abs(t_stat), df))


alpha = 0.025
# Calculate the critical t-value for alpha=0.05
t_critical = stats.t.ppf(1 - alpha/2, df)

print(f"t-statistic: {t_stat:.4f}")
print(f"Critical t-value: {t_critical:.4f}")
print(f"Degrees of freedom: {df}")
print(f"p-value: {p_value:.10f}")


# Check if difference is statistically significant
alpha = 0.05
if p_value < alpha:
    print(f"The difference is statistically significant (p < {alpha})")
else:
    print(f"The difference is not statistically significant (p >= {alpha})")

In [ ]:
import seaborn as sns

In [ ]:
palette = sns.color_palette("colorblind", 10)
import matplotlib.colors as mcolors
hexes = [mcolors.to_hex(p) for p in palette]
print(hexes)

In [ ]:
models_df.to_csv('Results_On_Everything.csv', index=False)

In [ ]:
import matplotlib.gridspec as gridspec

# Create a figure with a specific layout
fig = plt.figure(figsize=(8*1.3/1.5, 6.5*1.3/1.5))



# Create grid specification for the layout
gs = gridspec.GridSpec(2, 2, height_ratios=[1, 1], width_ratios=[1, 1])

# Create the first subplot (takes up the whole top row)
ax1 = fig.add_subplot(gs[0, :])

# Create the second and third subplots (bottom row)
ax2 = fig.add_subplot(gs[1, 0])
ax3 = fig.add_subplot(gs[1, 1])

# Plot 1: XGB model comparison (top)
plotted_system = models_df[(models_df['model_type'] == 'xgb')]

# Create the barplot for Plot 1
sns.barplot(
    data=plotted_system,
    x="extra",
    y="diff_mean",
    hue="features",
    palette='colorblind',
    errorbar=None,
    ax=ax1,
)


# Add labels and styling for Plot 1
ax1.set_xlabel('Exp. Data Points')
ax1.set_ylabel('Δ MAPE (%)')
ax1.grid(axis='y', linestyle='--', alpha=0.7)
ax1.set_title('XGB Performance')

# Get the legend for the first plot and set title
legend = ax1.legend(title="Descriptor Count", fontsize='small', title_fontsize='small')

# Plot 2: NN model comparison (bottom left)
plotted_system = models_df[(models_df['model_type'] == 'nn') & (models_df['features'] == 10)]

# Create the barplot for Plot 2
sns.barplot(
    data=plotted_system,
    x="extra",
    y="diff_mean",
    color=palette[0],
    errorbar=None,
    ax=ax2
)

# Add labels and styling for Plot 2
ax2.set_xlabel('Exp. Data Points')
ax2.set_ylabel('Δ MAPE (%)')
ax2.grid(axis='y', linestyle='--', alpha=0.7)
ax2.set_title('NN Performance')

# Plot 3: VAE model comparison (bottom right)
plotted_system = models_df[(models_df['model_type'] == 'vae') & (models_df['features'] == 10)]

# Create the barplot for Plot 3
sns.barplot(
    data=plotted_system,
    x="extra",
    y="diff_mean",
    color=palette[0],
    errorbar=None,
    ax=ax3
)

# Add labels and styling for Plot 3
ax3.set_xlabel('Exp. Data Points')
ax3.set_ylabel('Δ MAPE (%)')
ax3.grid(axis='y', linestyle='--', alpha=0.7)
ax3.set_title('VAE Performance')

plt.tight_layout()
plt.show()

In [ ]:
def viewGraph(systems : list, n, empirical_label='Empirical Data', x_label='Weight Fraction Solvent 1',ax=None):

    ja_model = JouybanAcreeModel()  
    x_values = np.linspace(0, 1, 101)

    
    # Create a new figure if ax is not provided
    if ax is None:
        fig, ax = plt.subplots(figsize=(16*1.3/3, 9*1.3/3))

    
    for system in systems:
        system_load = SystemDesign.load(system['filename'] + '.pkl')
        
        x,y = system_load.get_data_split_df()
        y_pred = system_load.predict_model(x)

        results_df = system_load.dataprocess.raw_data.merge(
            y_pred,
            left_index=True,
            right_index=True,
            suffixes=('','_pred')
        )
        
        JA_fit_pred = ja_model.predict(
            x_values, 
            results_df['solvent_1_pure'].iloc[n],
            results_df['solvent_2_pure'].iloc[n], 
            results_df['temperature'].iloc[n],
            results_df['J0_pred'].iloc[n],
            results_df['J1_pred'].iloc[n],
            results_df['J2_pred'].iloc[n],
        )
        
        ax.plot(x_values, JA_fit_pred, label=system['label'], color=system['color'], linestyle=system['dash'])
        
    # JA_fit_real = ja_model.predict(
    #     x_values, 
    #     results_df['solvent_1_pure'].iloc[n],
    #     results_df['solvent_2_pure'].iloc[n], 
    #     results_df['temperature'].iloc[n],
    #     results_df['J0'].iloc[n],
    #     results_df['J1'].iloc[n],
    #     results_df['J2'].iloc[n],
    # )

    # # Plot the JA model
    # plt.plot(x_values, JA_fit_real, label='Empirical JA Model', color=palette[0])


    group_index = int(results_df.iloc[n]['group_index'])
    group = ja_groups[group_index]
    
    ax.scatter(group['solvent_1_weight_fraction'], group['solubility_g_g'], color=palette[7], label=empirical_label)
    
    ax.set_xlabel(x_label)
    ax.set_ylabel('Solubility (g/g)')
    ax.legend()
    ax.grid(True)
    
    if ax is None:
        plt.show()
            
    return ax

In [ ]:
ss_df = models_df[models_df['combination'] == 'sst']

In [ ]:
# 387 for the artuneate system
# viewGraph('xgb_system_1_3_10_features', 387)
# 51 for sofo system
# viewGraph('xgb_system_1_3_10_features', 51)
# 2 for Iminodibenyl system
# viewGraph('xgb_system_1_3_10_features', 2)

In [ ]:
paper_dict = []
for key, values in models.items():
    sys = values['system']
    res = values['results']
    
    r = sys.evaluate_model()
    paper_dict.append({
        'key': key,
        'mae': r['mae'],
        'r2': r['r2'],      
    })

In [ ]:
from IPython.display import display, HTML

paper_df = pd.DataFrame(paper_dict)

paper_df[['model_type', 'combination', 'extra', 'features']] = paper_df['key'].str.extract(r'(\w+)_system(?:_([a-z]+))?_(\d+)_(\d+)_features')
    
paper_df.fillna('', inplace=True)
# Create a mapping dictionary
combination_map = {
    'sst': 'C',
    'ss': 'CT', 
    's': 'SCT',
    '': 'SSCT'
}

# Apply the mapping to the combination column
paper_df['combination'] = paper_df['combination'].map(lambda x: combination_map.get(x, x))
paper_df['model_type'] = paper_df['model_type'].map(lambda x: x.upper())

paper_df.sort_values(by='r2', ascending=False, inplace=True)

paper_df = paper_df[['model_type', 'combination', 'extra', 'features','mae','r2']]
# Rename columns before filtering
paper_df.rename(columns={
    'model_type': 'Model',
    'extra': 'Data Points',
    'features': 'Feature Count',
    'mae': 'MAE',
    'r2': 'R²',
    'combination': 'Encoded Var.'
}, inplace=True)

# Format the numeric columns
paper_df['MAE'] = paper_df['MAE'].map('{:.3f}'.format).astype(float)
paper_df['R²'] = paper_df['R²'].map('{:.3f}'.format).astype(float)
paper_df['Data Points'] = paper_df['Data Points'].astype(int)
paper_df['Feature Count'] = paper_df['Feature Count'].astype(int)


# Define a function to filter interesting models
def filter_interesting_models(df):
    # Create a new dataframe for interesting models
    interesting_models = pd.DataFrame()
    
    # 1. Include the best performing model overall (highest R² or lowest MAE)
    best_model = df.sort_values(by=['R²', 'MAE'], ascending=[False, True]).iloc[0:5]
    interesting_models = pd.concat([interesting_models, best_model])
    
    # 2. Include the best model of each type (VAE, NN, XGB)
    for model_type in ['VAE', 'NN', 'XGB']:
        best_of_type = df[df['Model'] == model_type].sort_values(by=['R²', 'MAE'], ascending=[False, True]).iloc[0:1]
        interesting_models = pd.concat([interesting_models, best_of_type])
    
    # 3. Include the best model for each encoding variable (CT, SCT, SSCT, C)
    for encoding in ['CT', 'SCT', 'SSCT', 'C']:
        best_of_encoding = df[df['Encoded Var.'] == encoding].sort_values(by=['R²', 'MAE'], ascending=[False, True]).iloc[0:1]
        interesting_models = pd.concat([interesting_models, best_of_encoding])
    
    # 4. Include models with different descriptor counts to show the effect
    for model_type in ['NN', 'XGB', 'VAE']:
        for encoding in ['CT']:
            for descriptor in [10, 50, 500]:
                best_of_desc = df[(df['Model'] == model_type) & 
                                 (df['Encoded Var.'] == encoding) & 
                                 (df['Feature Count'] == descriptor)].sort_values(by=['R²', 'MAE'], ascending=[False, True]).iloc[0:1]
                interesting_models = pd.concat([interesting_models, best_of_desc])
    
    # 5. Include one of the worst performing models for contrast
    worst_model = df[df['R²'] > 0].sort_values(by=['R²', 'MAE'], ascending=[True, False]).iloc[0:2]
    interesting_models = pd.concat([interesting_models, worst_model])
    
    # 6. Include one model with negative R² to show poor performance
    negative_r2 = df[df['R²'] < 0].sort_values(by=['R²'], ascending=[True]).iloc[0:1]
    interesting_models = pd.concat([interesting_models, negative_r2])
    
    # Remove duplicates and sort by performance
    interesting_models = interesting_models.drop_duplicates()
    
    interesting_models = interesting_models.sort_values(by=['Model','R²', 'MAE'], ascending=[False,False, True])
    
    return interesting_models

# Apply the filter and display the results
interesting_df = filter_interesting_models(paper_df)
print(f"Selected {len(interesting_df)} models out of {len(paper_df)} total models")

interesting_df['MAE'] = interesting_df['MAE'].apply(lambda x: f'{x:.3f}'[1:] if x > 0 else f'{x:.3f}')
interesting_df['R²'] = interesting_df['R²'].apply(lambda x: f'{x:.3f}'[1:] if x > 0 else '-' + f'{-x:.3f}'[1:])

tex = interesting_df.to_latex(
    index=False,
    float_format='%.3f',
    column_format='cccc||cc',
    escape=False,
    caption='Selected Models for Analysis based on the Test DataSet',
    label='tab:selected_models',
    position='htbp',
    header=['Model', 'Enc.', 'Ex.Pts', 'Feat.', 'MAE', '$R^2$']
)
display(HTML(f"<pre>{tex}</pre>"))

In [ ]:
interesting_df.to_clipboard(index=False, float_format='%.3f', header=['Model', 'Enc.', 'Ex.Pts', 'Feat.', 'MAE', '$R^2$'])

In [ ]:
tex2 = paper_df[paper_df['Model'] == 'XGB'].to_latex(
    index=False,
    float_format='%.3f',
    column_format='cccc||cc',
    escape=False,
    caption='Selected Models for Analysis',
    label='tab:selected_models',
    position='htbp',
    header=['Model', 'Enc.', 'Ex.Pts', 'Feat.', 'MAE', '$R^2$']
)
display(HTML(f"<pre>{tex2}</pre>"))

In [ ]:
systems = [
    {
        'label': 'NN JA Model',
        'filename': '../../output/models/nn_system_ss_0_10_features',
        'color': palette[1],
        'dash': '--',
    },
    {
        'label': 'VAE JA Model',
        'filename': '../../output/models/vae_system_ss_0_10_features',
        'color': palette[2],
        'dash': '-',
    },
    {
        'label': 'XGB JA Model',
        'filename': '../../output/models/xgb_system_s_0_10_features',
        'color': palette[3],
        'dash': '-.',
    }
]

In [ ]:
viewGraph(systems, 6,
          empirical_label='3-chloropyrazin-2-amine',
          x_label='Ethane-1,2-diol — Water Weight Fraction')


In [ ]:
viewGraph(systems, 51, empirical_label='Sofosbuvir Exp. Data',x_label='Acetone — Water Weight Fraction')
viewGraph(systems, 387, empirical_label='Artesunate Exp. Data',x_label='Propane-1,2-diol — Water Weight Fraction')
viewGraph(systems, 2, empirical_label='Iminodibenzyl Exp. Data',x_label='Acetone — Water Weight Fraction')

In [ ]:
import matplotlib.gridspec as gridspec

# Create a figure with a specific layout
fig = plt.figure(figsize=(16*1.3/1.5, 9*1.3/1.5))

# Create grid specification for the layout
gs = gridspec.GridSpec(2, 2, height_ratios=[1, 1], width_ratios=[1, 1])

# Create the first subplot (takes up the whole top row)
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])

# Create the second and third subplots (bottom row)
ax3 = fig.add_subplot(gs[1, 0])
ax4 = fig.add_subplot(gs[1, 1])

viewGraph(systems, 387, empirical_label='Artesunate Exp. Data',x_label='Propane-1,2-diol — Water Weight Fraction',ax=ax1)
viewGraph(systems, 51, empirical_label='Sofosbuvir Exp. Data',x_label='Acetone — Water Weight Fraction',ax=ax2)
# viewGraph(systems, 2, empirical_label='Iminodibenzyl Exp. Data',x_label='Acetone — Water Weight Fraction',ax=ax3)
viewGraph(systems, 40, empirical_label='Fenbufen Exp. Data',x_label='Acetone — Water Weight Fraction',ax=ax3)
viewGraph(systems, 6,empirical_label='3-chloropyrazin-2-amine', x_label='Ethane-1,2-diol — Water Weight Fraction',ax=ax4)

# Adjust y-axis scaling for each subplot
for ax in [ax1, ax3, ax4]:
    # Scale values by 1000
    formatter = plt.FuncFormatter(lambda x, pos: '{:.1f}'.format(x*1000))
    ax.yaxis.set_major_formatter(formatter)
    ax.set_ylabel("Solubility (mg/g)")


# Adjust layout
plt.tight_layout()

fig.show()

In [ ]:
xgbmodel : SystemDesign = SystemDesign.load('../../output/models/xgb_system_s_2_50_features.pkl')

importance = xgbmodel.model._get_feature_importances()
features = xgbmodel.model.selected_features

importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': importance
})

importance_df.sort_values(by='Importance', ascending=False, inplace=True)
# Drop rows where Feature contains "system_"
importance_df = importance_df[~importance_df['Feature'].str.contains('system_')]

# Show the top 15 features for better readability
importance_df.head(15)

In [ ]:
xgbmodel : SystemDesign = SystemDesign.load('../../output/models/xgb_system_ss_1_50_features.pkl')

importance = xgbmodel.model._get_feature_importances()
features = xgbmodel.model.selected_features

importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': importance
})

importance_df.sort_values(by='Importance', ascending=False, inplace=True)
# Drop rows where Feature contains "system_"
importance_df = importance_df[~importance_df['Feature'].str.contains('system_')]

# Show the top 15 features for better readability
importance_df.head(15)